In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import pymongo
from datetime import datetime
import pytz

from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder
from telegram.constants import ParseMode

from bot import config
from bot import database
from bot.app import show_balance_handle

In [ ]:
application = (
    ApplicationBuilder()
    .token(config.telegram_token)
    .build()
)

db = database.Database()

In [ ]:
newsletter_id = "text_improver_release"

db.create_newsletter(newsletter_id)
already_sent_to_user_ids = set(db.get_newsletter_attribute(newsletter_id, "already_sent_to_user_ids"))

print(f"Already sent to users: {already_sent_to_user_ids}")

In [ ]:
user_dicts = list(db.user_collection.find({"username": "karfly"}))
print(f"Found {len(user_dicts)} users")

In [ ]:
for user_dict in user_dicts:        
    if user_dict["_id"] in already_sent_to_user_ids:
        print(f"Skipping {user_dict['_id']}. Already sent before")
        continue
    
    text = "Meet the new chat mode – <b>📝 Text Improver</b>!\n"
    text += "Send a text, and the bot will correct spelling, fix mistakes and improve the text – just like Grammarly app.\n\n"
    text += "You can select the chat mode by sending the /mode <b>command</b>. Try it now!"
    
    try:
        await application.bot.send_message(
            user_dict['chat_id'],
            text,
            parse_mode=ParseMode.HTML
        )
        print(f"Successfully sent to {user_dict['_id']}")
              
        db.add_user_to_newsletter(newsletter_id, user_dict["_id"])
    except Exception as e:
        print(f"Failed to send message to {user_dict['_id']}. Reason: {e}")